In [15]:
import pandas as pd
from scipy.stats import wasserstein_distance
from statistics import mean
from tqdm import tqdm

### Lettura dati file degli score

In [16]:
#PATH_FILE_SCORE = './tabellaScoreNuova.csv'
PATH_FILE_SCORE = './scoreCsvDatasetFinale.csv'

scores = pd.read_csv(PATH_FILE_SCORE)
scores.head(10)


,Utente1,Utente2,ScoreML
0,AgatielloRoberto_000,AgatielloRoberto_001,0.526569
1,AgatielloRoberto_000,AgatielloRoberto_002,0.491637
2,AgatielloRoberto_000,AgatielloRoberto_003,0.525314
3,AgatielloRoberto_000,AgatielloRoberto_004,0.516008
4,AgatielloRoberto_000,AgatielloRoberto_005,0.567200
5,AgatielloRoberto_000,AgatielloRoberto_006,0.439096
6,AgatielloRoberto_000,AgatielloRoberto_007,0.454429
7,AgatielloRoberto_000,AgatielloRoberto_008,0.488878
8,AgatielloRoberto_000,AgatielloRoberto_009,0.418618
9,AgatielloRoberto_000,AgatielloRoberto_010,0.454418


In [17]:
scores = scores[scores.Utente2 != 'VenturaAlessio_000']

### Aggiunta colonne di utilità

In [18]:
def calcola_id_soggetto_da_utente(u):
    return u.split("_")[0]

def calcola_num_acquisizione_da_utente(u):
    return int(u.split("_")[1])

In [19]:
scores['Primo soggetto'] = scores['Utente1'].apply(lambda u: calcola_id_soggetto_da_utente(u))
scores['Secondo soggetto'] = scores['Utente2'].apply(lambda u: calcola_id_soggetto_da_utente(u))
scores['Numero prima acquisizione'] = scores['Utente1'].apply(lambda u: calcola_num_acquisizione_da_utente(u))
scores['Numero seconda acquisizione'] = scores['Utente2'].apply(lambda u: calcola_num_acquisizione_da_utente(u))

scores.head()

,Utente1,Utente2,ScoreML,Primo soggetto,Secondo soggetto,Numero prima acquisizione,Numero seconda acquisizione
0,AgatielloRoberto_000,AgatielloRoberto_001,0.526569,AgatielloRoberto,AgatielloRoberto,0,1
1,AgatielloRoberto_000,AgatielloRoberto_002,0.491637,AgatielloRoberto,AgatielloRoberto,0,2
2,AgatielloRoberto_000,AgatielloRoberto_003,0.525314,AgatielloRoberto,AgatielloRoberto,0,3
3,AgatielloRoberto_000,AgatielloRoberto_004,0.516008,AgatielloRoberto,AgatielloRoberto,0,4
4,AgatielloRoberto_000,AgatielloRoberto_005,0.567200,AgatielloRoberto,AgatielloRoberto,0,5


In [20]:
# a = scores.copy()
# del a['Numero prima acquisizione']
# del a['Numero seconda acquisizione']
# a = a.rename(columns={'Utente1':'Acquisizione1', 'Utente2':'Acquisizione2'})
# a.head(20)

### Calcolo delle pseudo-labels

In [21]:
def get_set_intra_classe(utente, scores):
    id_soggetto = calcola_id_soggetto_da_utente(utente)
    id_acquisizione = calcola_num_acquisizione_da_utente(utente)
    # Filtro per soggetto
    soggetto_scores = scores[(scores['Primo soggetto'] == id_soggetto) & (scores['Secondo soggetto'] == id_soggetto)]
    # Filtro per acquisizione
    soggetto_scores = soggetto_scores[(soggetto_scores['Numero prima acquisizione'] == id_acquisizione) | (soggetto_scores['Numero seconda acquisizione'] == id_acquisizione)]
    return set(soggetto_scores["ScoreML"])


def get_set_inter_classe(utente, scores):
    id_soggetto = calcola_id_soggetto_da_utente(utente)
    id_acquisizione = calcola_num_acquisizione_da_utente(utente)    
    
    id_soggetto = calcola_id_soggetto_da_utente(utente)
    id_acquisizione = calcola_num_acquisizione_da_utente(utente)
    # Filtro per soggetto
    soggetto_scores = scores[((scores['Primo soggetto'] == id_soggetto) & (scores['Secondo soggetto'] != id_soggetto)) | ((scores['Primo soggetto'] != id_soggetto) & (scores['Secondo soggetto'] == id_soggetto))]
    # Filtro per acquisizione
    soggetto_scores = soggetto_scores[(soggetto_scores['Utente1'] == utente) | (soggetto_scores['Utente2'] == utente)]
    return set(soggetto_scores["ScoreML"])
    

In [22]:
TRADE_OFF_PARAMETER = 1

lista_utenti = list(set(scores.Utente1.unique()) | set(scores.Utente2.unique()))
lista_utenti.sort()

In [23]:
df_labels = pd.DataFrame(columns=['Acquisizione', 'Label']) 

for utente in tqdm(lista_utenti):
    set_intra_classe = get_set_intra_classe(utente, scores)
    QxP = mean(set_intra_classe)

    set_inter_classe = get_set_inter_classe(utente, scores)
    list_inter_classe = list(set_inter_classe)
    list_inter_classe.sort(reverse=True)
    list_confronto_intra_classe = list_inter_classe[0: len(set_intra_classe)]
    QxN = wasserstein_distance(list(set_intra_classe), list_confronto_intra_classe)

    label = QxP + TRADE_OFF_PARAMETER * QxN

    row_acquisizione = pd.DataFrame.from_dict({'Acquisizione': utente, 'Label': label}, orient='index').T
    df_labels = pd.concat([df_labels, row_acquisizione])


100%|██████████| 718/718 [00:35<00:00, 20.21it/s]


### Normalizzazione delle labels tra 0 e 100

In [24]:
min_label = df_labels.Label.min()
max_label = df_labels.Label.max()
df_labels['Label_norm'] = df_labels['Label'].apply(lambda l: 100 * (l - min_label)/(max_label - min_label))
df_labels = df_labels.reset_index(drop=True)

In [25]:
df_labels.sort_values(by=['Label'])

,Acquisizione,Label,Label_norm
133,CovielloChristian_004,0.135723,0.000000
611,SpadolaGianfranco_001,0.212083,7.504152
649,TozziVito_002,0.214847,7.775687
610,SpadolaGianfranco_000,0.235944,9.848961
369,MoccardiFrancesco_000,0.246236,10.860381
...,...,...,...
21,AnastasioMarco_000,1.113998,96.137237
19,AlberucciAlessandro_001,1.123472,97.068275
159,DefilippisGiuseppe_002,1.137007,98.398337
22,AnastasioMarco_001,1.147208,99.400844


### Analisi su singola acquisizione

In [28]:
acquisizioneAnalisi = 'SpadolaGianfranco_001'
test_df = scores[(scores.Utente1 == acquisizioneAnalisi) | (scores.Utente2 == acquisizioneAnalisi)]
test_df.sort_values(by='ScoreML').head()

,Utente1,Utente2,ScoreML,Primo soggetto,Secondo soggetto,Numero prima acquisizione,Numero seconda acquisizione
87193,CovielloChristian_004,SpadolaGianfranco_001,0.057013,CovielloChristian,SpadolaGianfranco,4,1
18235,BellafanteValentina_001,SpadolaGianfranco_001,0.061400,BellafanteValentina,SpadolaGianfranco,1,1
37897,CanestriChristian_002,SpadolaGianfranco_001,0.062491,CanestriChristian,SpadolaGianfranco,2,1
74092,ColangeloFederico_002,SpadolaGianfranco_001,0.075670,ColangeloFederico,SpadolaGianfranco,2,1
172522,LleshiXhoandri_002,SpadolaGianfranco_001,0.080899,LleshiXhoandri,SpadolaGianfranco,2,1


In [29]:
df_labels.to_csv('./tabellaPseudoLabelsDatasetCompleto.csv', index=False)

In [23]:
df_labels[['Acquisizione', 'Label_norm']].head(20)

,Acquisizione,Label_norm
0,1_M_1,64.257481
1,1_M_10,73.084762
2,1_M_11,72.182680
3,1_M_12,73.886902
4,1_M_13,68.552254
5,1_M_14,73.473366
6,1_M_15,76.472275
7,1_M_2,70.799711
8,1_M_3,74.963246
9,1_M_4,73.781919
